In [1]:
# Tahap 1: Persiapan Data
# Kumpulkan Data:

# Kumpulkan data kamera yang mencakup berbagai fitur seperti harga, resolusi, ukuran sensor, berat, brand, dan lain-lain.
# Kumpulkan juga data preferensi pengguna (jika ada) yang dapat membantu dalam proses pembobotan.
# Preprocessing Data:

# Bersihkan data dari outliers atau data yang tidak lengkap.
# Normalisasi atau skala data jika diperlukan.

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Membaca data dari CSV
data = pd.read_csv('Data_Camera.csv')

# Menampilkan beberapa baris pertama untuk verifikasi
print("Data awal:")
print(data.head())

# Menghapus tanda pemisah ribuan dan mengubah tipe data menjadi float
data['Price'] = data['Price'].str.replace(',', '').astype(float)

# Menampilkan kolom Price untuk verifikasi
print("\nKolom Price setelah konversi:")
print(data['Price'].head())

# Pastikan kolom lain yang diperlukan juga dalam format numerik
data['Jenis Sensor'] = pd.to_numeric(data['Jenis Sensor'], errors='coerce')
data['Color Depth'] = pd.to_numeric(data['Color Depth'], errors='coerce')
data['LowLight ISO'] = pd.to_numeric(data['LowLight ISO'], errors='coerce')

# Menampilkan data setelah konversi untuk verifikasi
print("\nData setelah konversi ke numerik:")
print(data[['Jenis Sensor', 'Color Depth', 'LowLight ISO']].head())

# Menghapus baris yang memiliki nilai NaN (jika ada)
data.dropna(subset=['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO'], inplace=True)

# Menampilkan data setelah menghapus baris dengan nilai NaN
print("\nData setelah menghapus NaN:")
print(data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']].head())

# Memastikan bahwa ada data yang tersisa untuk dinormalisasi
if not data.empty:
    # Normalisasi data
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']])

    # Memasukkan kembali data yang telah dinormalisasi ke dalam DataFrame
    data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']] = data_scaled

    # Menampilkan data untuk memverifikasi hasil
    print("\nData setelah normalisasi:")
    print(data.head())
else:
    print("\nTidak ada data yang tersisa setelah pembersihan. Silakan periksa data input Anda.")


Data awal:
                 Nama Kamera Brand Kamera Jenis Sensor  Color Depth  \
0             Canon EOS 300D        Canon        APS-C         21.0   
1              Canon EOS 10D        Canon        APS-C         21.0   
2         Nikon Coolpix P340        Nikon       1/1.7"         20.7   
3         Nikon Coolpix P330        Nikon       1/1.7"         21.0   
4  Panasonic Lumix DMC FX150    Panasonic       1/1.7"         18.4   

   Dynamic Range  LowLight ISO      Price   Preferensi  
0           10.8           544  5,500,000  Profesional  
1           10.9           571  1,200,000         Hobi  
2           11.9           273  2,838,000       Pemula  
3           11.7           213  1,150,000       Pemula  
4            9.6           101  3,100,000         Hobi  

Kolom Price setelah konversi:
0    5500000.0
1    1200000.0
2    2838000.0
3    1150000.0
4    3100000.0
Name: Price, dtype: float64

Data setelah konversi ke numerik:
   Jenis Sensor  Color Depth  LowLight ISO
0       

In [3]:
# Tahap 2: Tentukan Kriteria dan Subkriteria
# 1. Identifikasi Kriteria Utama
    # Kriteria utama adalah faktor-faktor utama yang mempengaruhi keputusan pembelian kamera. Misalnya, beberapa kriteria utama bisa meliputi:

    # Harga
    # Jenis Sensor
    # Resolusi
    # Kedalaman Warna
    # ISO Rendah
# 2. Tentukan Subkriteria (jika diperlukan)
    # Subkriteria adalah rincian lebih lanjut dari kriteria utama. Tidak semua kriteria memerlukan subkriteria, tetapi untuk beberapa faktor yang lebih kompleks, subkriteria dapat membantu memperjelas keputusan. Misalnya:

    # Harga:
    # Harga di bawah $500
    # Harga antara $500 - $1000
    # Harga di atas $1000
    # Jenis Sensor:
    # APS-C
    # Full Frame
    # Medium Format
# 3. Struktur Hierarki
    # Struktur hierarki menunjukkan bagaimana kriteria dan subkriteria diorganisir. Berikut adalah contoh struktur hierarki untuk keputusan pembelian kamera:

In [4]:
#                       Tujuan
#                         |
#            ---------------------------
#            |           |             |
#          Harga      Jenis Sensor   Resolusi   ... (dan seterusnya)
#            |           |             |
#   -----------       --------       --------
#   |    |    |       |      |       |      |
#  < $500  $500-$1000 > $1000 APS-C Full Frame ... (dan seterusnya)


In [5]:
# Implementasi dengan Python
# Untuk menerapkan metode F-AHP dalam Python, kita perlu membangun matriks perbandingan 
# berpasangan untuk setiap level dalam hierarki. 
# Di sini, kita menggunakan library seperti numpy untuk menangani perhitungan matriks.

In [6]:
# Tentukan kriteria utama
kriteria = ['Price', 'Jenis Sensor', 'Color Depth', 'Dynamic Range', 'LowLight ISO']

# Matriks perbandingan berpasangan untuk kriteria utama
# Contoh skala perbandingan berpasangan (misal, 1 = sama penting, 3 = cukup lebih penting, 5 = sangat lebih penting, dst.)
# Anggap kita sudah menentukan perbandingan kriteria ini sebelumnya
pairwise_matrix = np.array([
    [1, 3, 5, 7, 9],
    [1/3, 1, 3, 5, 7],
    [1/5, 1/3, 1, 3, 5],
    [1/7, 1/5, 1/3, 1, 3],
    [1/9, 1/7, 1/5, 1/3, 1]
])

# Fungsi untuk menghitung bobot dari matriks perbandingan berpasangan
def calculate_weights(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)].real
    weights = max_eigenvector / np.sum(max_eigenvector)
    return weights

# Hitung bobot kriteria utama
kriteria_weights = calculate_weights(pairwise_matrix)
print("Bobot Kriteria Utama:", dict(zip(kriteria, kriteria_weights)))

# Jika kita memiliki subkriteria, kita juga perlu membuat matriks perbandingan berpasangan untuk subkriteria setiap kriteria.
# Misalnya untuk subkriteria Harga:
subkriteria_harga = ['< 500.000', '500.000-1.000.000', '> 1.000.000']
pairwise_matrix_harga = np.array([
    [1, 2, 3],
    [1/2, 1, 2],
    [1/3, 1/2, 1]
])

# Hitung bobot subkriteria Harga
subkriteria_harga_weights = calculate_weights(pairwise_matrix_harga)
print("Bobot Subkriteria Harga:", dict(zip(subkriteria_harga, subkriteria_harga_weights)))

Bobot Kriteria Utama: {'Price': 0.5128128127814086, 'Jenis Sensor': 0.2614990557217761, 'Color Depth': 0.12897642312026958, 'Dynamic Range': 0.06337652767438601, 'LowLight ISO': 0.033335180702159815}
Bobot Subkriteria Harga: {'< 500.000': 0.5396145502210747, '500.000-1.000.000': 0.29696133121249724, '> 1.000.000': 0.16342411856642797}


In [7]:
# Tahap 3: Fuzzy AHP
    # Langkah-langkah Fuzzy AHP
    # Definisikan Skala Fuzzy:

    # Tentukan skala perbandingan fuzzy menggunakan bilangan fuzzy segitiga (Triangular Fuzzy Numbers - TFN).
    # Buat Matriks Perbandingan Fuzzy:

    # Bangun matriks perbandingan berpasangan dengan nilai fuzzy.
    # Hitung Sintesis Fuzzy:

    # Hitung bobot prioritas fuzzy untuk setiap kriteria dan subkriteria.
    # Defuzzifikasi:

    # Konversi bilangan fuzzy menjadi nilai tunggal (crisp) untuk mendapatkan bobot akhir.
# Langkah 1: Definisikan Skala Fuzzy
    # Berikut adalah contoh skala fuzzy dengan bilangan fuzzy segitiga:

    # 1: (1, 1, 1)
    # 3: (2, 3, 4)
    # 5: (4, 5, 6)
    # 7: (6, 7, 8)
    # 9: (8, 9, 10)
# Langkah 2: Buat Matriks Perbandingan Fuzzy
# Bangun matriks perbandingan berpasangan untuk kriteria dan subkriteria menggunakan skala fuzzy yang telah ditentukan.


In [8]:
# Definisikan bilangan fuzzy segitiga
fuzzy_scale = {
    1: (1, 1, 1),
    3: (2, 3, 4),
    5: (4, 5, 6),
    7: (6, 7, 8),
    9: (8, 9, 10)
}

# Matriks perbandingan berpasangan fuzzy (contoh)
fuzzy_pairwise_matrix = [
    [(1, 1, 1), (2, 3, 4), (4, 5, 6)],
    [(1/4, 1/3, 1/2), (1, 1, 1), (2, 3, 4)],
    [(1/6, 1/5, 1/4), (1/4, 1/3, 1/2), (1, 1, 1)]
]

# Convert to numpy array for easier manipulation
fuzzy_pairwise_matrix = np.array(fuzzy_pairwise_matrix, dtype=object)

In [9]:
# Langkah 3: Hitung Sintesis Fuzzy
# Untuk menghitung bobot prioritas fuzzy, kita perlu melakukan operasi fuzzy pada matriks.

In [10]:
import numpy as np
from functools import reduce

# Definisikan bilangan fuzzy segitiga
fuzzy_scale = {
    1: (1, 1, 1),
    3: (2, 3, 4),
    5: (4, 5, 6),
    7: (6, 7, 8),
    9: (8, 9, 10)
}

# Matriks perbandingan berpasangan fuzzy (contoh)
fuzzy_pairwise_matrix = [
    [(1, 1, 1), (2, 3, 4), (4, 5, 6)],
    [(1/4, 1/3, 1/2), (1, 1, 1), (2, 3, 4)],
    [(1/6, 1/5, 1/4), (1/4, 1/3, 1/2), (1, 1, 1)]
]

# Convert to numpy array for easier manipulation
fuzzy_pairwise_matrix = np.array(fuzzy_pairwise_matrix, dtype=object)

# Function to perform fuzzy multiplication
def fuzzy_multiply(a, b):
    return (a[0] * b[0], a[1] * b[1], a[2] * b[2])

# Function to perform fuzzy addition
def fuzzy_add(a, b):
    return (a[0] + b[0], a[1] + b[1], a[2] + b[2])

# Compute fuzzy sum for each row
fuzzy_sums = [reduce(fuzzy_add, row) for row in fuzzy_pairwise_matrix]

# Normalize the fuzzy sums
total_sum = reduce(fuzzy_add, fuzzy_sums)

# Normalize each fuzzy sum
def normalize_fuzzy(sums, total_sum):
    return (sums[0] / total_sum[2], sums[1] / total_sum[1], sums[2] / total_sum[0])

fuzzy_extent = [normalize_fuzzy(sums, total_sum) for sums in fuzzy_sums]

# Langkah 4: Defuzzifikasi
# Defuzzifikasi mengubah bilangan fuzzy menjadi nilai crisp. Salah satu metode defuzzifikasi adalah metode centroid.

# Defuzzify fuzzy weights using centroid method
def defuzzify(triangular_fuzzy_number):
    l, m, u = triangular_fuzzy_number
    return (l + m + u) / 3

# Defuzzify fuzzy weights
crisp_weights = [defuzzify(weight) for weight in fuzzy_extent]

print("Crisp Weights:")
print(crisp_weights)


Crisp Weights:
[0.6439333175373473, 0.31366352794573205, 0.11025486140972351]


In [11]:
# Penjelasan
# Definisikan Skala Fuzzy:

# Menggunakan bilangan fuzzy segitiga untuk mewakili perbandingan kriteria.
# Buat Matriks Perbandingan Fuzzy:

# Bangun matriks perbandingan berpasangan dengan nilai fuzzy.
# Hitung Sintesis Fuzzy:

# Menggunakan operasi fuzzy untuk menghitung bobot prioritas fuzzy.
# Defuzzifikasi:

# Mengonversi bobot fuzzy menjadi nilai crisp untuk mendapatkan bobot akhir.

In [12]:
# Tahap 4: Integrasi dengan Machine Learning
# Latih Model Machine Learning:
# Gunakan model machine learning untuk memprediksi preferensi pengguna atau menyarankan kamera berdasarkan kriteria yang telah dihitung.

In [13]:
# Periksa apakah data kosong
if data.empty:
    print("Dataset kosong.")

# Periksa apakah kolom yang diperlukan ada
required_columns = ['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO', 'Preferensi']
missing_columns = [col for col in required_columns if col not in data.columns]
if missing_columns:
    print(f"Kolom yang hilang: {missing_columns}")

# Periksa nilai kosong
null_values = data[required_columns].isnull().sum()
print(f"Nilai kosong dalam kolom:\n{null_values}")

# Periksa beberapa baris pertama dari DataFrame
print(data.head())

# Lanjutkan dengan train_test_split jika semua pemeriksaan lolos
if not data.empty and not missing_columns and null_values.sum() == 0:
    X = data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']]
    y = data['Preferensi']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # Latih model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
else:
    print("Data tidak cocok untuk dibagi dan dilatih.")


Dataset kosong.
Nilai kosong dalam kolom:
Price           0.0
Jenis Sensor    0.0
Color Depth     0.0
LowLight ISO    0.0
Preferensi      0.0
dtype: float64
Empty DataFrame
Columns: [Nama Kamera, Brand Kamera, Jenis Sensor, Color Depth, Dynamic Range, LowLight ISO, Price, Preferensi]
Index: []
Data tidak cocok untuk dibagi dan dilatih.


In [14]:

# Split data
X = data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']]
y = data['Preferensi']  # Kolom preferensi harus ada dalam data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.